# Final Project Technical Notebook

Name(s): Bilal Najar, Kevin Akman
Date: 11/28/2023

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import math
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KernelDensity
import seaborn as sns
import random
from scipy.stats import norm, dgamma, geom, t



In [4]:
# grab local dataset
import os

heart_df = pd.read_csv(r'heart.csv')
heart_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Encoding categorical variables
label_encoders = {}
for col in heart_df.columns:
    le = LabelEncoder()
    heart_df[col] = le.fit_transform(heart_df[col])
    label_encoders[col] = le

# Splitting data
X = heart_df.drop('HeartDisease', axis=1)
y = heart_df['HeartDisease']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=27)

# Initialize and train the logistic regression model
logistic_model = LogisticRegression(max_iter=1000, random_state=27)
logistic_model.fit(X_train, y_train)

# Generating predictions
y_pred = logistic_model.predict(X_test)
y_pred = logistic_model.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.87
              precision    recall  f1-score   support

           0       0.86      0.85      0.85       126
           1       0.87      0.88      0.88       150

    accuracy                           0.87       276
   macro avg       0.87      0.86      0.86       276
weighted avg       0.87      0.87      0.87       276
